<a href="https://colab.research.google.com/github/erendagasan/Eren-Dagasan-Personal/blob/main/damper%20-%20molecular%20-%20laminar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [38]:
#LAMINAR CALCULATION
life_time_final = 60 * 60 * 24 * 30 * 12 * 50
time_step = 300
Pinitial = 2000
Pambient = 1000
V = 0.001
Pcritic = 1500
Qinitial_vals = np.arange(1.1E-9, 1E-8, 1e-9)
# Qinitial_vals = np.arange(1.1E-8, 1E-7, 1E-8)
time_actual = np.arange(time_step, life_time_final, time_step)

real_df_list = []
calc_df_list = []

for Qinitial in Qinitial_vals:
    Premain_start = Pinitial - (Qinitial * time_step / V)
    Q_start = (Qinitial * ((Premain_start**2) - (Pambient**2)) / ((Pinitial**2) - (Pambient**2)))
    Pambient_start = Pambient+Pinitial-Premain_start

    Premain_list = []
    Premain_list.append(Pinitial)
    Premain_list.append(Premain_start)

    Pambient_list = []
    Pambient_list.append(Pambient)
    Pambient_list.append(Pambient_start)

    Q_calc_future_list = []
    Q_calc_future_list.append(Qinitial)
    Q_calc_future_list.append(Q_start)

    Premain_calc_buffer = Premain_start
    Premain_calc_buffer_before = Premain_start
    Q_buffer_calc = Q_start

    i = 0
    while Premain_calc_buffer > Pcritic:
        Premain_calc_buffer = Premain_calc_buffer - (Q_buffer_calc * time_step / V)
        Premain_list.append(Premain_calc_buffer)

        Pambient_list.append(Pambient)

        calc = Q_buffer_calc * ((Premain_calc_buffer**2) - (Pambient**2)) / ((Premain_calc_buffer_before**2) - (Pambient**2))
        Q_calc_future_list.append(calc)
        Q_buffer_calc = calc

        Premain_calc_buffer_before = Premain_calc_buffer
        i+=1


    Premain_real_list = []
    Premain_real_list.append(Pinitial)
    Premain_real_list.append(Premain_start)

    Pambient_real_list = []
    Pambient_real_list.append(Pambient)
    Pambient_real_list.append(Pambient_start)

    Q_real_future_list = []
    Q_real_future_list.append(Qinitial)
    Q_real_future_list.append(Q_start)

    Premain_real_buffer = Premain_start
    Premain_real_buffer_before = Premain_start
    Pambient_real_buffer = Pambient_start
    Pambient_real_buffer_before = Pambient_start
    Q_buffer_real = Q_start

    i = 0
    while Premain_real_buffer > Pcritic and Pambient_real_buffer < Pcritic:
        Premain_real_buffer = Premain_real_buffer - (Q_buffer_real * time_step / V)
        Premain_real_list.append(Premain_real_buffer)

        Pambient_real_buffer = Pambient + (Pinitial-Premain_real_buffer)
        Pambient_real_list.append(Pambient_real_buffer)

        calc = Q_buffer_real * ((Premain_real_buffer**2) - (Pambient_real_buffer_before**2)) / ((Premain_real_buffer_before**2) - (Pambient_real_buffer_before**2))
        Q_real_future_list.append(calc)
        Q_buffer_real = calc

        Premain_real_buffer_before = Premain_real_buffer
        Pambient_real_buffer_before = Pambient_real_buffer
        i+=1

    data_real = {
        "Qreal": Q_real_future_list,
        "Penv_real": Pambient_real_list,
        "Pint_real": Premain_real_list,
        "Duration": time_actual[:len(Premain_real_list)]
    }

    data_real = pd.DataFrame(data_real)

    data_real['Duration_min']   = data_real['Duration']/60
    data_real['Duration_hour']  = data_real['Duration_min']/60
    data_real['Duration_day']   = data_real['Duration_hour']/24
    data_real['Duration_month'] = data_real['Duration_day']/30
    data_real['Duration_year']  = data_real['Duration_month']/12


    real_df_list.append(data_real)

    data_calc = {
        "Qcalc": Q_calc_future_list,
        "Penv_calc": Pambient_list,
        "Pint_calc": Premain_list,
        "Duration": time_actual[:len(Premain_list)]
    }

    data_calc['Duration_min']   = data_calc['Duration']/60
    data_calc['Duration_hour']  = data_calc['Duration_min']/60
    data_calc['Duration_day']   = data_calc['Duration_hour']/24
    data_calc['Duration_month'] = data_calc['Duration_day']/30
    data_calc['Duration_year']  = data_calc['Duration_month']/12

    data_calc = pd.DataFrame(data_calc)
    calc_df_list.append(data_calc)

In [39]:
duration_df = pd.DataFrame()

for index, Qinitial in enumerate(Qinitial_vals):
  duration_df = pd.concat([duration_df, pd.DataFrame([[Qinitial, round(real_df_list[index]["Duration_year"].iloc[-1], 2), round(calc_df_list[index]["Duration_year"].iloc[-1], 2)]], columns=["Qinitial", "Real Duration (Year)", "Calculated Duration (Year)"])])

duration_df.reset_index(drop=True, inplace=True)
duration_df

,Qinitial,Real Duration (Year),Calculated Duration (Year)
0,1.100000e-09,32.70,22.40
1,2.100000e-09,17.13,11.73
2,3.100000e-09,11.60,7.95
3,4.100000e-09,8.77,6.01
4,5.100000e-09,7.05,4.83
5,6.100000e-09,5.90,4.04
6,7.100000e-09,5.07,3.47
7,8.100000e-09,4.44,3.04
8,9.100000e-09,3.95,2.71


In [40]:
#MOLECULAR FLOW CALCULATION

life_time_final = 60 * 60 * 24 * 30 * 12 * 50
time_step = 300
Pinitial = 2000
Pambient = 1000
V = 0.001
Pcritic = 1500
Qinitial_vals = np.arange(1.1E-9, 1E-8, 1e-9)
# Qinitial_vals = np.arange(1.1E-8, 1E-7, 1E-8)
time_actual = np.arange(time_step, life_time_final, time_step)

real_df_list = []
calc_df_list = []

for Qinitial in Qinitial_vals:
    Premain_start = Pinitial - (Qinitial * time_step / V)
    Q_start = (Qinitial * ((Premain_start) - (Pambient)) / ((Pinitial) - (Pambient)))
    Pambient_start = Pambient+Pinitial-Premain_start

    Premain_list = []
    Premain_list.append(Pinitial)
    Premain_list.append(Premain_start)

    Pambient_list = []
    Pambient_list.append(Pambient)
    Pambient_list.append(Pambient_start)

    Q_calc_future_list = []
    Q_calc_future_list.append(Qinitial)
    Q_calc_future_list.append(Q_start)

    Premain_calc_buffer = Premain_start
    Premain_calc_buffer_before = Premain_start
    Q_buffer_calc = Q_start

    i = 0
    while Premain_calc_buffer > Pcritic:
        Premain_calc_buffer = Premain_calc_buffer - (Q_buffer_calc * time_step / V)
        Premain_list.append(Premain_calc_buffer)

        Pambient_list.append(Pambient)

        calc = Q_buffer_calc * ((Premain_calc_buffer) - (Pambient)) / ((Premain_calc_buffer_before) - (Pambient))
        Q_calc_future_list.append(calc)
        Q_buffer_calc = calc

        Premain_calc_buffer_before = Premain_calc_buffer
        i+=1


    Premain_real_list = []
    Premain_real_list.append(Pinitial)
    Premain_real_list.append(Premain_start)

    Pambient_real_list = []
    Pambient_real_list.append(Pambient)
    Pambient_real_list.append(Pambient_start)

    Q_real_future_list = []
    Q_real_future_list.append(Qinitial)
    Q_real_future_list.append(Q_start)

    Premain_real_buffer = Premain_start
    Premain_real_buffer_before = Premain_start
    Pambient_real_buffer = Pambient_start
    Pambient_real_buffer_before = Pambient_start
    Q_buffer_real = Q_start

    i = 0
    while Premain_real_buffer > Pcritic and Pambient_real_buffer < Pcritic:
        Premain_real_buffer = Premain_real_buffer - (Q_buffer_real * time_step / V)
        Premain_real_list.append(Premain_real_buffer)

        Pambient_real_buffer = Pambient + (Pinitial-Premain_real_buffer)
        Pambient_real_list.append(Pambient_real_buffer)

        calc = Q_buffer_real * ((Premain_real_buffer) - (Pambient_real_buffer_before)) / ((Premain_real_buffer_before) - (Pambient_real_buffer_before))
        Q_real_future_list.append(calc)
        Q_buffer_real = calc

        Premain_real_buffer_before = Premain_real_buffer
        Pambient_real_buffer_before = Pambient_real_buffer
        i+=1

    data_real = {
        "Qreal": Q_real_future_list,
        "Penv_real": Pambient_real_list,
        "Pint_real": Premain_real_list,
        "Duration": time_actual[:len(Premain_real_list)]
    }

    data_real = pd.DataFrame(data_real)

    data_real['Duration_min']   = data_real['Duration']/60
    data_real['Duration_hour']  = data_real['Duration_min']/60
    data_real['Duration_day']   = data_real['Duration_hour']/24
    data_real['Duration_month'] = data_real['Duration_day']/30
    data_real['Duration_year']  = data_real['Duration_month']/12


    real_df_list.append(data_real)

    data_calc = {
        "Qcalc": Q_calc_future_list,
        "Penv_calc": Pambient_list,
        "Pint_calc": Premain_list,
        "Duration": time_actual[:len(Premain_list)]
    }

    data_calc['Duration_min']   = data_calc['Duration']/60
    data_calc['Duration_hour']  = data_calc['Duration_min']/60
    data_calc['Duration_day']   = data_calc['Duration_hour']/24
    data_calc['Duration_month'] = data_calc['Duration_day']/30
    data_calc['Duration_year']  = data_calc['Duration_month']/12

    data_calc = pd.DataFrame(data_calc)
    calc_df_list.append(data_calc)

In [41]:
duration_df = pd.DataFrame()

for index, Qinitial in enumerate(Qinitial_vals):
  duration_df = pd.concat([duration_df, pd.DataFrame([[Qinitial, round(real_df_list[index]["Duration_year"].iloc[-1], 2), round(calc_df_list[index]["Duration_year"].iloc[-1], 2)]], columns=["Qinitial", "Real Duration (Year)", "Calculated Duration (Year)"])])

duration_df.reset_index(drop=True, inplace=True)
duration_df

,Qinitial,Real Duration (Year),Calculated Duration (Year)
0,1.100000e-09,29.23,20.26
1,2.100000e-09,15.31,10.61
2,3.100000e-09,10.37,7.19
3,4.100000e-09,7.84,5.44
4,5.100000e-09,6.30,4.37
5,6.100000e-09,5.27,3.65
6,7.100000e-09,4.53,3.14
7,8.100000e-09,3.97,2.75
8,9.100000e-09,3.53,2.45


In [ ]:
import plotly.graph_objs as go
import plotly.io as pio

fig = go.Figure()

fig.add_trace(go.Scatter(x=duration_df['Real Duration (Year)'], y=duration_df['Qinitial'], mode='lines',
                         name='Real Duration (Year)', line=dict(width=2)))
fig.add_trace(go.Scatter(x=duration_df['Calculated Duration (Year)'], y=duration_df['Qinitial'], mode='lines',
                         name='Calculated Duration (Year)', line=dict(width=2)))

#fig.add_trace(go.Scatter(x=duration_df['Real Duration (Year)'], y=duration_df['Qinitial'],
#                         mode='markers', name='Real Duration (Year)',))
#fig.add_trace(go.Scatter(x=duration_df['Calculated Duration (Year)'], y=duration_df['Qinitial'],
#                         mode='markers', name='Calculated Duration (Year)'))

fig.update_layout(
    title="Calculated and Real Durations for Qinitial Values",
    xaxis_title='Duration (Year)',
    yaxis_title='Qinitial',
    yaxis=dict(tickformat=".2e"),
    legend=dict(x=1.05, y=1),
)

pio.write_html(fig, file="duration_comparison.html")
fig.show()

In [8]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

trace1_1 = go.Scatter(x=calc_df_list[0]["Duration_year"], y=calc_df_list[0]["Qcalc"], name=f"Calculated for {Qinitial_vals[0]}", mode="lines", line=dict(width=2, dash="dash"))
trace1_2 = go.Scatter(x=real_df_list[0]["Duration_year"], y=real_df_list[0]["Qreal"], name=f"Real for {Qinitial_vals[0]}", mode="lines", line=dict(width=2))

trace2_1 = go.Scatter(x=calc_df_list[1]["Duration_year"], y=calc_df_list[1]["Qcalc"], name=f"Calculated for {Qinitial_vals[1]}", line=dict(width=2, dash="dash"))
trace2_2 = go.Scatter(x=real_df_list[1]["Duration_year"], y=real_df_list[1]["Qreal"], name=f"Real for {Qinitial_vals[1]}", mode="lines", line=dict(width=2))

fig = make_subplots(rows=1, cols=2, subplot_titles=(Qinitial_vals[0], Qinitial_vals[1]))
fig.add_trace(trace1_1, row=1, col=1)
fig.add_trace(trace1_2, row=1, col=1)
fig.add_trace(trace2_1, row=1, col=2)
fig.add_trace(trace2_2, row=1, col=2)

fig.update_layout(
    title="2 Different Qinitial Value",
    yaxis_title="Leakage (Q)",
    legend=dict(x=1.05, y=1)
)

fig.update_xaxes(title="Duration (Years)")
fig.update_xaxes(title="Duration (Years)")
fig.update_yaxes(tickformat=".2e", row=1, col=1)
fig.update_yaxes(tickformat=".2e", row=1, col=2)

# pio.write_html(fig, file="subplots.html")
fig.show()


In [ ]:
import plotly.graph_objs as go
import plotly.io as pio

traces = []

for index, (Qinitial, real_df) in enumerate(zip(Qinitial_vals, real_df_list)):
    duration = real_df["Duration_year"]
    Qreal = real_df["Qreal"]

    trace = go.Scatter(
        x=duration,
        y=Qreal,
        mode='lines',
        name=f"{Qinitial}",
        line=dict(width=2),
    )

    traces.append(trace)

    fig = go.Figure(data=traces)

fig.update_layout(
    title="Real Leakage Lines for Qinitial Values",
    xaxis_title="Duration (Years)",
    yaxis_title="Qreal",
    yaxis=dict(tickformat=".2e"),
    legend=dict(x=1.05, y=1),  # Adjust the legend position
)

pio.write_html(fig, file="real_leakage_lines.html")
#fig.show()

In [ ]:
import plotly.graph_objs as go
import plotly.io as pio

traces = []

for index, (Qinitial, calc_df) in enumerate(zip(Qinitial_vals, calc_df_list)):
    duration = calc_df["Duration_year"]
    Qcalc = calc_df["Qcalc"]

    trace = go.Scatter(
        x=duration,
        y=Qcalc,
        mode='lines',
        name=f"{Qinitial}",
        line=dict(width=2),
    )

    traces.append(trace)

fig = go.Figure(data=traces)

fig.update_layout(
    title="Calculated Leakage Lines for Qinitial Values",
    xaxis_title="Duration (Year)",
    yaxis_title="Qcalc",
    yaxis=dict(tickformat=".2e"),
    legend=dict(x=1.05, y=1),
)

pio.write_html(fig, file="calculated_leakage_lines.html")
#fig.show()


In [ ]:
import plotly.graph_objs as go
import plotly.io as pio

traces = []

for index, (Qinitial, calc_df) in enumerate(zip(Qinitial_vals, calc_df_list)):
    duration = calc_df["Duration_year"]
    Qcalc = calc_df["Pint_calc"]

    trace = go.Scatter(
        x=duration,
        y=Qcalc,
        mode='lines',
        name=f"{Qinitial}",
        line=dict(width=2),
    )

    traces.append(trace)

fig = go.Figure(data=traces)

fig.update_layout(
    title="Calculated P Internal Lines for Qinitial Values",
    xaxis_title="Duration (Year)",
    yaxis_title="Pressure",
    yaxis=dict(tickformat=".2e"),
    legend=dict(x=1.05, y=1),
)

pio.write_html(fig, file="premain_calc_lines.html")
#fig.show()


In [ ]:
import plotly.graph_objs as go
import plotly.io as pio

traces = []

for index, (Qinitial, real_df) in enumerate(zip(Qinitial_vals, real_df_list)):
    duration = real_df["Duration_year"]
    Qreal = real_df["Pint_real"]

    trace = go.Scatter(
        x=duration,
        y=Qreal,
        mode='lines',
        name=f"{Qinitial}",
        line=dict(width=2),
    )

    traces.append(trace)

    fig = go.Figure(data=traces)

fig.update_layout(
    title="Real P Internal Lines for Qinitial Values",
    xaxis_title="Duration (Years)",
    yaxis_title="Pressure",
    yaxis=dict(tickformat=".2e"),
    legend=dict(x=1.05, y=1),  # Adjust the legend position
)

pio.write_html(fig, file="premain_real_lines.html")
#fig.show()